<a href="https://colab.research.google.com/github/chirayu-khandelwal/Parkinson_Detection/blob/main/PHD_PD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PhD-Level Approach: Maximizing Parkinson's Severity Prediction Performance

**Objective:** To rigorously explore and optimize machine learning methodologies for predicting `motor_UPDRS` score using the UCI Parkinson's Telemonitoring dataset, employing advanced techniques in feature engineering, modeling, validation, and interpretation.

**Methodology:** This notebook outlines a comprehensive, research-oriented workflow, acknowledging the limitation of using pre-extracted features instead of raw audio signals.

**Workflow Overview:**
1.  **Setup & Literature Review:** Define environment, tools, and review relevant studies.
2.  **Deep Data Exploration & Understanding:** Go beyond basic EDA.
3.  **Advanced Preprocessing & Cleaning:** Robust handling of potential issues.
4.  **Extensive Feature Engineering & Exploration:** Explore diverse feature types beyond basic FFT.
5.  **Sophisticated Feature Selection & Dimensionality Reduction:** Select the most informative feature subset.
6.  **Rigorous Modeling Strategy & Validation Design:** Plan advanced models and robust validation (Nested CV).
7.  **Advanced Hyperparameter Optimization:** Employ efficient search strategies (e.g., Bayesian Optimization).
8.  **Model Training & Evaluation (Nested CV):** Execute the defined strategy.
9.  **Ensemble Methods & Final Model Selection:** Combine strong models.
10. **Model Interpretation & Explainability:** Understand *why* the model predicts.
11. **Reporting, Discussion & Future Work:** Document findings, limitations, and next steps.

## 1. Setup & Literature Review

In [ ]:
# --- Core Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore') # Suppress routine warnings

# --- Data Handling ---
# Install ucimlrepo if not already installed
try:
    import ucimlrepo
except ImportError:
    print("Installing ucimlrepo...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ucimlrepo"])
    import ucimlrepo

from ucimlrepo import fetch_ucirepo

# --- Preprocessing & Feature Engineering ---
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.impute import KNNImputer # More advanced imputation
from numpy.fft import fft, fftfreq

!pip install PyWavelets
import pywt # For Wavelet Transforms

from scipy import stats # For statistical features

# --- Feature Selection ---
from sklearn.feature_selection import RFE, SelectFromModel, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV # Lasso for feature selection

# --- Modeling ---
# Standard
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor

!pip install xgboost
import xgboost as xgb

!pip install lightgbm
import lightgbm as lgb

# Deep Learning
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Other Advanced
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

# --- Validation & Hyperparameter Tuning ---
from sklearn.model_selection import KFold, GroupKFold, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV
!pip install scikit-optimize # Or hyperopt, optuna
from skopt import BayesSearchCV # Example: Bayesian Optimization
from skopt.space import Real, Categorical, Integer

# --- Ensemble Methods ---
from sklearn.ensemble import VotingRegressor, StackingRegressor

# --- Evaluation & Interpretation ---
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
# !pip install shap
import shap

# --- Configuration ---
SEED = 42
N_SPLITS_OUTER = 5 # For outer cross-validation loop
N_SPLITS_INNER = 3 # For inner hyperparameter tuning loop (within outer loop)
N_ITER_BAYES = 50 # Number of iterations for Bayesian optimization
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Libraries imported.")

### 1b. Literature Review (Conceptual)

*   **Goal:** Understand state-of-the-art methods for UPDRS prediction specifically from *voice features* similar to those in this dataset.
*   **Keywords:** Parkinson's UPDRS prediction, voice features, machine learning, regression, jitter, shimmer, NHR, HNR, dysphonia.
*   **Sources:** Google Scholar, PubMed, IEEE Xplore, arXiv.
*   **Key Questions:**
    *   Which specific voice features are consistently found to be most predictive?
    *   What feature engineering techniques (beyond basic stats) have been applied?
    *   Which ML/DL models perform best?
    *   What validation strategies are used (simple split, CV, subject-aware CV)?
    *   What performance metrics (RMSE, R², MAE) are typically reported?
    *   What are the reported limitations and challenges?
*   **Action:** Summarize findings to guide feature selection, model choice, and interpretation.

## 2. Deep Data Exploration & Understanding

In [ ]:
print("--- 2a. Loading Data ---")
try:
    parkinsons_telemonitoring = fetch_ucirepo(id=189)
    df_full = parkinsons_telemonitoring.data.original # Get original df which might include subject#
    if df_full is None:
       print("Original dataframe not available, fetching features/targets separately.")
       X_data = parkinsons_telemonitoring.data.features
       y_data = parkinsons_telemonitoring.data.targets
       df_full = pd.concat([X_data, y_data], axis=1)
    print("Dataset loaded.")
    # print(parkinsons_telemonitoring.metadata)
    # print(parkinsons_telemonitoring.variables)
except Exception as e:
    print(f"Error loading data: {e}. Attempting local CSV.")
    try:
        df_full = pd.read_csv('data.csv')
        print("Loaded from data.csv")
    except FileNotFoundError:
        print("ERROR: data.csv not found.")
        raise

target_variable = 'motor_UPDRS'
if target_variable not in df_full.columns:
    raise ValueError(f"Target variable '{target_variable}' not found.")

print(f"Full dataset shape: {df_full.shape}")
display(df_full.head())

In [ ]:
print("\n--- 2b. Deep Exploration ---")

# 1. Basic Info
print("\nBasic Info:")
df_full.info()

# 2. Descriptive Statistics
print("\nDescriptive Statistics:")
display(df_full.describe().T)

# 3. Missing Values (Re-check)
print("\nMissing Values:")
print(df_full.isna().sum())
# If missing values are found, consider advanced imputation like KNNImputer later.

# 4. Duplicate Rows
print("\nDuplicate Rows:")
print(f"Number of duplicate rows: {df_full.duplicated().sum()}")
# Consider implications: Are duplicates expected? Errors?
# df_full = df_full.drop_duplicates()

# 5. Target Variable Distribution
print(f"\nTarget Variable ({target_variable}) Distribution:")
sns.histplot(df_full[target_variable], kde=True)
plt.title(f'Distribution of {target_variable}')
plt.show()
sns.boxplot(x=df_full[target_variable])
plt.title(f'Boxplot of {target_variable}')
plt.show()
print(f"Skewness: {df_full[target_variable].skew():.2f}")
# Consider potential transformations (log, Box-Cox) if highly skewed, though often not needed for tree models.

# 6. Feature Distributions & Outliers
print("\nFeature Distributions (Example - first few numerical):")
numerical_features = df_full.select_dtypes(include=np.number).drop(columns=[target_variable, 'total_UPDRS'], errors='ignore')
if 'subject#' in numerical_features.columns: numerical_features = numerical_features.drop(columns=['subject#'])

plt.figure(figsize=(15, min(5*len(numerical_features.columns)//4, 30)))
for i, col in enumerate(numerical_features.columns[:min(len(numerical_features.columns), 16)]): # Plot first N features
    plt.subplot(4, 4, i + 1)
    sns.histplot(numerical_features[col], kde=True, bins=30)
    plt.title(col)
plt.tight_layout()
plt.show()
# Consider outlier detection methods (IQR, Z-score, Isolation Forest) and handling strategies.

# 7. Correlations
print("\nCorrelation Analysis:")
plt.figure(figsize=(18, 15))
corr_matrix = df_full.drop(columns=['subject#'], errors='ignore').corr()
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, fmt='.1f') # Annot=True is too dense
plt.title('Feature Correlation Matrix')
plt.show()

target_corr = corr_matrix[[target_variable]].sort_values(by=target_variable, ascending=False)
plt.figure(figsize=(6, 10))
sns.heatmap(target_corr, annot=True, cmap='viridis', fmt='.2f')
plt.title(f'Feature Correlation with {target_variable}')
plt.show()
# Note high correlations between features (multicollinearity) - may affect linear models.

# 8. Subject Analysis (if 'subject#' exists and is reliable)
if 'subject#' in df_full.columns:
    print("\nSubject Analysis:")
    n_subjects = df_full['subject#'].nunique()
    print(f"Number of unique subjects: {n_subjects}")
    subject_counts = df_full['subject#'].value_counts()
    print("Records per subject (summary):")
    print(subject_counts.describe())
    plt.figure(figsize=(10, 4))
    sns.histplot(subject_counts, bins=30)
    plt.title('Distribution of Records per Subject')
    plt.xlabel('Number of Records')
    plt.show()
    # This unequal distribution is important for validation (GroupKFold).

    # Check variation of target within/between subjects (example)
    plt.figure(figsize=(15, 6))
    sns.boxplot(x='subject#', y=target_variable, data=df_full, order=subject_counts.index[:min(n_subjects, 20)]) # Show first ~20 subjects
    plt.xticks(rotation=90)
    plt.title(f'{target_variable} Distribution Across Subjects (Sample)')
    plt.show()
    # Significant variability suggests subject ID could be a useful feature or grouping factor.


## 3. Advanced Preprocessing & Cleaning

In [ ]:
print("--- 3. Advanced Preprocessing ---")

# Define Features (X) and Target (y) for modeling
y = df_full[target_variable].values
X = df_full.drop(columns=[target_variable, 'total_UPDRS'], errors='ignore') # Drop both targets
groups = None # For GroupKFold if subject ID is used
if 'subject#' in X.columns:
    print("Using 'subject#' for grouped cross-validation.")
    groups = X['subject#'].values
    X = X.drop(columns=['subject#']) # Drop identifier after extracting groups
else:
    print("Warning: 'subject#' column not found. Standard KFold will be used.")

feature_names_initial = X.columns.tolist()
print(f"Shape of X before further processing: {X.shape}")
print(f"Shape of y: {y.shape}")

# --- Outlier Handling (Example using IQR - apply cautiously) ---
# This step is complex and domain-knowledge dependent. Excessive removal can harm performance.
# Consider robust models or robust scaling instead.
# Example (conceptual - would need careful implementation within CV):
# for col in X.columns:
#     Q1 = X[col].quantile(0.25)
#     Q3 = X[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     # Option 1: Cap outliers
#     # X[col] = np.clip(X[col], lower_bound, upper_bound)
#     # Option 2: Mark outliers (requires model that can handle NaN or specific value)
#     # X[col+'_is_outlier'] = ((X[col] < lower_bound) | (X[col] > upper_bound)).astype(int)
print("Outlier handling strategy: To be applied carefully, potentially using RobustScaler or within CV.")

# --- Advanced Imputation (Example: KNNImputer if missing values exist) ---
if X.isna().sum().sum() > 0:
   print("Applying KNNImputer...")
   imputer = KNNImputer(n_neighbors=5)
   X_imputed = imputer.fit_transform(X)
   X = pd.DataFrame(X_imputed, columns=feature_names_initial)
else:
   print("No missing values found, skipping imputation.")

# --- Scaling Strategy ---
# Scaling will be applied *within* the cross-validation loop to prevent data leakage.
# Options: MinMaxScaler, StandardScaler, RobustScaler (good for outliers).
print("Scaling: To be performed within cross-validation using StandardScaler or RobustScaler.")


## 4. Extensive Feature Engineering & Exploration

In [ ]:
print("--- 4. Feature Engineering (Conceptual - apply within CV) ---")

# NOTE: Feature engineering should ideally be done *inside* the cross-validation loop
#       on the training fold only to prevent data leakage. This section defines the functions.

def engineer_features(X_train_fold, X_test_fold):
    """Applies various feature engineering techniques."""
    X_train_eng = X_train_fold.copy()
    X_test_eng = X_test_fold.copy()
    original_cols = X_train_fold.columns.tolist()

    # --- 4a. Polynomial / Interaction Features ---
    # Caution: Can lead to very high dimensionality
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    # Fit only on training data
    poly.fit(X_train_eng[original_cols])
    X_train_poly = poly.transform(X_train_eng[original_cols])
    X_test_poly = poly.transform(X_test_eng[original_cols])
    poly_feature_names = poly.get_feature_names_out(original_cols)
    X_train_eng = pd.DataFrame(X_train_poly, columns=poly_feature_names, index=X_train_eng.index)
    X_test_eng = pd.DataFrame(X_test_poly, columns=poly_feature_names, index=X_test_eng.index)
    print(f"Shape after PolynomialFeatures: {X_train_eng.shape}")

    # --- 4b. Statistical Features (Examples) ---
    # Rolling window features could be attempted if data had a reliable time order, but likely not applicable here.
    # Higher-order moments (applied per feature - less common unless theoretically justified)
    # for col in original_cols:
    #    X_train_eng[f'{col}_skew'] = stats.skew(X_train_eng[col]) # This is a single value, not row-wise
    #    X_test_eng[f'{col}_skew'] = stats.skew(X_test_eng[col]) # Need careful application

    # --- 4c. Frequency Domain Features (FFT Variants) ---
    # Apply FFT to each column *signal* across the fold (as done before, but inside CV)
    # Consider: Energy in frequency bands, spectral entropy, phase info (complex)
    n_fft_components = 5
    for col in original_cols:
        # Fit FFT summary stats ON TRAINING FOLD ONLY
        signal_train = X_train_eng[col].values
        N_train = len(signal_train)
        if N_train <= 1: continue
        yf_train = fft(signal_train)
        yf_mag_train = np.abs(yf_train[:N_train//2])
        if len(yf_mag_train) > 0:
            actual_n = min(n_fft_components, len(yf_mag_train))
            top_indices = np.argsort(yf_mag_train)[::-1][:actual_n]
            top_magnitudes = yf_mag_train[top_indices]
            padded_mags = np.pad(top_magnitudes, (0, n_fft_components - len(top_magnitudes)), 'constant') if len(top_magnitudes) < n_fft_components else top_magnitudes
            for i in range(n_fft_components):
                # Add the *summary* stat (calculated on train) to both train and test folds
                fft_val = padded_mags[i]
                X_train_eng[f'{col}_fft_mag_{i+1}'] = fft_val
                X_test_eng[f'{col}_fft_mag_{i+1}'] = fft_val # Use train-fold stat for test fold
        else:
             for i in range(n_fft_components):
                 X_train_eng[f'{col}_fft_mag_{i+1}'] = 0
                 X_test_eng[f'{col}_fft_mag_{i+1}'] = 0

    # --- 4d. Time-Frequency Domain Features (Wavelets) ---
    # Apply DWT to each column *signal* (similar limitations as FFT)
    # Example: Extract energy from DWT coefficients
    # wavelet = 'db4' # Example wavelet
    # for col in original_cols:
    #    coeffs_train = pywt.wavedec(X_train_eng[col], wavelet, level=3) # Example level
    #    # Extract features from coeffs_train (e.g., energy, entropy)
    #    # Apply the same *type* of feature extraction conceptually to test set
    #    # This requires careful thought on how to handle train/test consistency
    print("Wavelet feature engineering: Conceptually possible but needs careful implementation within CV.")

    # --- 4e. Domain-Specific Features (Based on Literature Review) ---
    # Example: Calculate ratios identified as important, e.g., Shimmer/Jitter ratio
    # if 'Shimmer(%)' in X_train_eng.columns and 'Jitter(%)' in X_train_eng.columns:
    #    X_train_eng['Shim_Jit_Ratio'] = X_train_eng['Shimmer(%)'] / (X_train_eng['Jitter(%)'] + 1e-6)
    #    X_test_eng['Shim_Jit_Ratio'] = X_test_eng['Shimmer(%)'] / (X_test_eng['Jitter(%)'] + 1e-6)

    # --- Final Check ---
    # Ensure no NaN/inf values introduced
    X_train_eng.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test_eng.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Impute any NaNs possibly created during engineering (using mean/median fitted on train)
    for col in X_train_eng.columns:
        if X_train_eng[col].isna().any():
            fill_value = X_train_eng[col].median()
            X_train_eng[col].fillna(fill_value, inplace=True)
            X_test_eng[col].fillna(fill_value, inplace=True) # Use train median for test

    return X_train_eng, X_test_eng

# Conceptual call - Actual call happens inside CV loop
# X_train_sample = pd.DataFrame(np.random.rand(100, len(feature_names_initial)), columns=feature_names_initial)
# X_test_sample = pd.DataFrame(np.random.rand(20, len(feature_names_initial)), columns=feature_names_initial)
# X_train_eng_sample, X_test_eng_sample = engineer_features(X_train_sample, X_test_sample)
# print(f"Sample engineered train shape: {X_train_eng_sample.shape}")

## 5. Sophisticated Feature Selection & Dimensionality Reduction

In [ ]:
print("--- 5. Feature Selection / Dimensionality Reduction (Conceptual - apply within CV) ---")

# NOTE: This step follows Feature Engineering and precedes Scaling/Modeling inside the CV loop.
#       It must be fitted *only* on the training fold.

def select_features(X_train_fold, y_train_fold, X_test_fold, method='lasso', n_features=50):
    """Selects features using a specified method."""
    print(f"Applying feature selection method: {method}...")

    if method == 'lasso':
        # Use LassoCV to find optimal alpha and select features
        # Scale data first for Lasso
        scaler_fs = StandardScaler().fit(X_train_fold)
        X_train_scaled = scaler_fs.transform(X_train_fold)
        X_test_scaled = scaler_fs.transform(X_test_fold)

        selector = SelectFromModel(LassoCV(cv=3, random_state=SEED, n_jobs=-1), threshold=1e-5) # Small threshold
        selector.fit(X_train_scaled, y_train_fold)
        X_train_selected = selector.transform(X_train_scaled)
        X_test_selected = selector.transform(X_test_scaled)
        selected_mask = selector.get_support()
        selected_features = X_train_fold.columns[selected_mask].tolist()
        print(f"Lasso selected {len(selected_features)} features.")
        # Return unscaled selected data
        return X_train_fold[selected_features], X_test_fold[selected_features]

    elif method == 'rfe':
        # Recursive Feature Elimination with a base estimator (e.g., RandomForest)
        # computationally expensive
        estimator = RandomForestRegressor(n_estimators=50, random_state=SEED, n_jobs=-1)
        selector = RFE(estimator, n_features_to_select=n_features, step=0.1) # Adjust step
        selector.fit(X_train_fold, y_train_fold)
        selected_mask = selector.get_support()
        selected_features = X_train_fold.columns[selected_mask].tolist()
        print(f"RFE selected {len(selected_features)} features.")
        return X_train_fold[selected_features], X_test_fold[selected_features]

    elif method == 'mutual_info':
        mi = mutual_info_regression(X_train_fold, y_train_fold, random_state=SEED)
        mi_series = pd.Series(mi, index=X_train_fold.columns).sort_values(ascending=False)
        selected_features = mi_series.head(n_features).index.tolist()
        print(f"Mutual Info selected {len(selected_features)} features.")
        return X_train_fold[selected_features], X_test_fold[selected_features]

    elif method == 'pca':
        # Dimensionality reduction rather than selection
        scaler_pca = StandardScaler().fit(X_train_fold)
        X_train_scaled = scaler_pca.transform(X_train_fold)
        X_test_scaled = scaler_pca.transform(X_test_fold)

        pca = PCA(n_components=n_features) # Or variance explained e.g., 0.95
        pca.fit(X_train_scaled)
        X_train_selected = pca.transform(X_train_scaled)
        X_test_selected = pca.transform(X_test_scaled)
        print(f"PCA applied, resulting shape: {X_train_selected.shape}")
        # Note: Feature names are lost ('PC1', 'PC2', ...)
        return pd.DataFrame(X_train_selected), pd.DataFrame(X_test_selected)

    else: # No selection
        return X_train_fold, X_test_fold

# Conceptual Call Example:
# X_train_sel, X_test_sel = select_features(X_train_eng_sample, y_train_sample, X_test_eng_sample, method='lasso')
# print(f"Sample selected train shape: {X_train_sel.shape}")

## 6. Rigorous Modeling Strategy & Validation Design

In [ ]:
print("--- 6. Modeling Strategy and Validation Design ---")

# --- Validation Strategy: Nested Cross-Validation ---
# Outer loop: Estimates generalization performance unbiasedly.
# Inner loop: Used for hyperparameter tuning for the model trained on the outer fold.
if groups is not None:
    # Use GroupKFold if subject IDs are available to prevent subject leakage
    outer_cv = GroupKFold(n_splits=N_SPLITS_OUTER)
    inner_cv = GroupKFold(n_splits=N_SPLITS_INNER) # Use GroupKFold here too if possible
    cv_groups = groups
    print(f"Using GroupKFold with {N_SPLITS_OUTER} outer splits and {N_SPLITS_INNER} inner splits.")
else:
    # Use standard KFold if no groups
    outer_cv = KFold(n_splits=N_SPLITS_OUTER, shuffle=True, random_state=SEED)
    inner_cv = KFold(n_splits=N_SPLITS_INNER, shuffle=True, random_state=SEED)
    cv_groups = None # No groups passed to splitting method
    print(f"Using KFold with {N_SPLITS_OUTER} outer splits and {N_SPLITS_INNER} inner splits.")

# --- Scoring Metric ---
# Use negative RMSE because optimization functions typically maximize
scoring = make_scorer(lambda y_true, y_pred: -np.sqrt(mean_squared_error(y_true, y_pred)))
print(f"Primary scoring metric: Negative RMSE")

# --- Candidate Models & Parameter Spaces (Examples) ---
models_to_evaluate = {
    'Ridge': {
        'model': RidgeCV(alphas=np.logspace(-4, 4, 9), cv=inner_cv), # RidgeCV does its own inner CV
        'param_space': None # Handled by RidgeCV
    },
    'Lasso': {
        'model': LassoCV(cv=inner_cv, random_state=SEED, n_jobs=-1, max_iter=2000),
        'param_space': None
    },
    'SVR': {
        'model': SVR(),
        'param_space': {
            'C': Real(1e-2, 1e3, prior='log-uniform'),
            'gamma': Real(1e-4, 1e1, prior='log-uniform'),
            'kernel': Categorical(['rbf']), # RBF is common, linear could be added
            'epsilon': Real(0.01, 0.5, prior='uniform')
        }
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=SEED, n_jobs=-1),
        'param_space': {
            'n_estimators': Integer(100, 1000),
            'max_depth': Integer(5, 50),
            'min_samples_split': Integer(2, 20),
            'min_samples_leaf': Integer(1, 10),
            'max_features': Real(0.1, 1.0, prior='uniform') # Fraction of features
        }
    },
    'XGBoost': {
        'model': xgb.XGBRegressor(random_state=SEED, objective='reg:squarederror', n_jobs=-1),
        'param_space': {
            'n_estimators': Integer(100, 1000),
            'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'max_depth': Integer(3, 15),
            'subsample': Real(0.5, 1.0, prior='uniform'),
            'colsample_bytree': Real(0.5, 1.0, prior='uniform'),
            'gamma': Real(0, 5, prior='uniform'), # Min loss reduction
            'reg_alpha': Real(1e-3, 1e2, prior='log-uniform'), # L1 reg
            'reg_lambda': Real(1e-3, 1e2, prior='log-uniform') # L2 reg
        }
    },
    'LightGBM': {
        'model': lgb.LGBMRegressor(random_state=SEED, n_jobs=-1),
        'param_space': {
            'n_estimators': Integer(100, 1500),
            'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
            'num_leaves': Integer(10, 100),
            'max_depth': Integer(5, 30),
            'subsample': Real(0.5, 1.0, prior='uniform'),
            'colsample_bytree': Real(0.5, 1.0, prior='uniform'),
            'reg_alpha': Real(1e-3, 1e2, prior='log-uniform'),
            'reg_lambda': Real(1e-3, 1e2, prior='log-uniform')
        }
    }
    # Add Deep Learning models here requires separate handling within the loop
    # Add GaussianProcessRegressor if dataset size permits (computationally heavy)
}

print(f"Defined {len(models_to_evaluate)} candidate models/parameter spaces.")

## 7. Advanced Hyperparameter Optimization

In [ ]:
print("--- 7. Hyperparameter Optimization Strategy ---")
print(f"Using Bayesian Optimization (BayesSearchCV) with {N_ITER_BAYES} iterations within the inner CV loop.")
# The actual optimization happens inside the nested CV loop in the next step.

## 8. Model Training & Evaluation (Nested CV)

In [ ]:
print("--- 8. Executing Nested Cross-Validation --- ")

outer_results = {}
best_models_per_fold = {}
oof_predictions = {} # Store out-of-fold predictions for ensembling/analysis
oof_indices = {}

# Define parameters for feature engineering and selection to test
feature_engineering_method = None # Or 'polynomial', 'fft_enhanced', etc. based on exploration
feature_selection_method = 'lasso' # Or 'rfe', 'mutual_info', None
n_features_to_select = 50 # If using selection method requiring N

# --- Outer Loop ---
for i, (train_outer_idx, test_outer_idx) in enumerate(outer_cv.split(X, y, groups=cv_groups)):
    print(f"\n--- Outer Fold {i+1}/{N_SPLITS_OUTER} ---")
    X_train_outer, X_test_outer = X.iloc[train_outer_idx], X.iloc[test_outer_idx]
    y_train_outer, y_test_outer = y[train_outer_idx], y[test_outer_idx]
    groups_outer_train = groups[train_outer_idx] if groups is not None else None

    # Store indices for OOF predictions
    current_fold_indices = X.iloc[test_outer_idx].index.tolist()

    # 1. Feature Engineering (Applied to this specific outer fold)
    # X_train_outer_eng, X_test_outer_eng = engineer_features(X_train_outer, X_test_outer)
    # For now, using original features + placeholder for selection
    X_train_outer_eng, X_test_outer_eng = X_train_outer, X_test_outer

    # 2. Feature Selection (Applied to this outer fold)
    # X_train_outer_sel, X_test_outer_sel = select_features(
    #     X_train_outer_eng, y_train_outer, X_test_outer_eng,
    #     method=feature_selection_method, n_features=n_features_to_select
    # )
    # Using engineered (currently same as original) features for now
    X_train_outer_sel, X_test_outer_sel = X_train_outer_eng, X_test_outer_eng

    # 3. Scaling (Fit on this outer train fold)
    # scaler = StandardScaler() # Or RobustScaler()
    scaler = RobustScaler() # Good choice given potential outliers
    X_train_outer_scaled = scaler.fit_transform(X_train_outer_sel)
    X_test_outer_scaled = scaler.transform(X_test_outer_sel)
    print(f" Fold {i+1}: Data shapes after FeatEng/Select/Scale - Train: {X_train_outer_scaled.shape}, Test: {X_test_outer_scaled.shape}")

    outer_results[f'Fold_{i+1}'] = {}
    best_models_per_fold[f'Fold_{i+1}'] = {}
    oof_predictions[f'Fold_{i+1}'] = {}
    oof_indices[f'Fold_{i+1}'] = current_fold_indices

    # --- Inner Loop (Hyperparameter Tuning) ---
    for model_name, config in models_to_evaluate.items():
        print(f"  Tuning {model_name}...")
        model = config['model']
        param_space = config['param_space']

        start_tune_time = time.time()

        if param_space: # Use Bayesian Optimization if space is defined
            # Need to ensure inner split uses groups correctly if outer uses them
            inner_cv_fold = inner_cv.split(X_train_outer_scaled, y_train_outer, groups=groups_outer_train)

            opt = BayesSearchCV(
                estimator=model,
                search_spaces=param_space,
                scoring=scoring,
                cv=inner_cv_fold, # Pass the generator
                n_iter=N_ITER_BAYES,
                random_state=SEED,
                n_jobs=-1, # Use multiple cores
                verbose=0 # Set to 1 for more details
            )
            try:
               opt.fit(X_train_outer_scaled, y_train_outer) # Fit the optimizer
               best_model_inner = opt.best_estimator_
               best_score_inner = opt.best_score_
            except Exception as e:
               print(f"    ERROR tuning {model_name}: {e}. Skipping.")
               best_model_inner = model # Fallback to default
               best_score_inner = -np.inf # Indicate failure

        elif hasattr(model, 'fit') and hasattr(model, 'predict'): # Models like RidgeCV/LassoCV handle their own tuning
             try:
                 model.fit(X_train_outer_scaled, y_train_outer)
                 best_model_inner = model # The fitted model is the 'best'
                 # Estimate inner score (approximate, as CV is internal)
                 y_pred_inner = best_model_inner.predict(X_train_outer_scaled)
                 best_score_inner = scoring._score_func(y_train_outer, y_pred_inner) # Score on training set (not ideal)
             except Exception as e:
                print(f"    ERROR fitting {model_name}: {e}. Skipping.")
                best_model_inner = model # Fallback
                best_score_inner = -np.inf # Indicate failure
        else:
            print(f"    Skipping tuning for {model_name} (no param space or not applicable).")
            best_model_inner = model # Use default
            best_score_inner = -np.inf # Indicate no tuning done

        tune_time = time.time() - start_tune_time

        # 4. Evaluate the *best model found in the inner loop* on the outer test set
        if best_score_inner > -np.inf: # Check if tuning/fitting was successful
           y_pred_outer = best_model_inner.predict(X_test_outer_scaled)
           outer_mse = mean_squared_error(y_test_outer, y_pred_outer)
           outer_rmse = np.sqrt(outer_mse)
           outer_mae = mean_absolute_error(y_test_outer, y_pred_outer)
           outer_r2 = r2_score(y_test_outer, y_pred_outer)

           print(f"    {model_name} - Outer Test RMSE: {outer_rmse:.4f}, R2: {outer_r2:.4f} (Tuning Time: {tune_time:.1f}s)")
           outer_results[f'Fold_{i+1}'][model_name] = {'RMSE': outer_rmse, 'MAE': outer_mae, 'R2': outer_r2}
           best_models_per_fold[f'Fold_{i+1}'][model_name] = best_model_inner
           oof_predictions[f'Fold_{i+1}'][model_name] = y_pred_outer
        else:
            print(f"    {model_name} - Skipped evaluation due to tuning/fitting error.")
            outer_results[f'Fold_{i+1}'][model_name] = {'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan}

    # --- Optional: Train/Evaluate Deep Learning Model Separately within Outer Fold ---
    # DL models often require different HPO (Keras Tuner) and training procedures
    # Placeholder:
    print("  Placeholder for DL model training/evaluation within this outer fold...")
    # 1. Reshape data: X_train_dl = X_train_outer_scaled.reshape(...), X_test_dl = X_test_outer_scaled.reshape(...)
    # 2. Define Keras Tuner search (e.g., Hyperband, BayesianOptimization)
    # 3. tuner.search(X_train_dl, y_train_outer, validation_split=0.2, callbacks=[...])
    # 4. best_dl_model = tuner.get_best_models(num_models=1)[0]
    # 5. y_pred_outer_dl = best_dl_model.predict(X_test_dl)
    # 6. Calculate metrics (RMSE, MAE, R2)
    # 7. Store results in outer_results, best_models_per_fold, oof_predictions

print("\nNested Cross-Validation finished.")

In [ ]:
# --- Aggregate Nested CV Results ---
aggregated_results = {}
for model_name in models_to_evaluate.keys(): # Add DL model name if implemented
    model_results = {'RMSE': [], 'MAE': [], 'R2': []}
    for fold in outer_results:
        if model_name in outer_results[fold]:
            model_results['RMSE'].append(outer_results[fold][model_name]['RMSE'])
            model_results['MAE'].append(outer_results[fold][model_name]['MAE'])
            model_results['R2'].append(outer_results[fold][model_name]['R2'])
        # else: Handle case where model failed in a fold

    if model_results['RMSE']: # If results exist for this model
       aggregated_results[model_name] = {
           'RMSE_mean': np.nanmean(model_results['RMSE']),
           'RMSE_std': np.nanstd(model_results['RMSE']),
           'MAE_mean': np.nanmean(model_results['MAE']),
           'R2_mean': np.nanmean(model_results['R2'])
       }

nested_cv_results_df = pd.DataFrame(aggregated_results).T
nested_cv_results_df = nested_cv_results_df.sort_values(by='RMSE_mean', ascending=True)

print("\n--- Aggregated Nested Cross-Validation Results (Mean +/- Std across outer folds) ---")
display(nested_cv_results_df.round(4))

# --- Reconstruct OOF Predictions ---
oof_preds_final = {}
true_values_oof = pd.Series(index=X.index, dtype=float)
model_names_with_preds = list(models_to_evaluate.keys()) # Add DL model name

for model_name in model_names_with_preds:
    oof_preds_final[model_name] = pd.Series(index=X.index, dtype=float)

for fold in oof_indices:
    indices = oof_indices[fold]
    true_values_oof.loc[indices] = y[indices] # Store true y for these indices
    for model_name in model_names_with_preds:
        if model_name in oof_predictions[fold]:
             oof_preds_final[model_name].loc[indices] = oof_predictions[fold][model_name].flatten()

# Drop rows where prediction might be missing (if a model failed in a fold)
oof_df = pd.DataFrame(oof_preds_final)
oof_df['True'] = true_values_oof
oof_df.dropna(inplace=True) # Drop rows where any model failed

print(f"\nReconstructed Out-of-Fold (OOF) predictions for {len(oof_df)} instances.")
# Can calculate overall OOF metrics here
print("Overall OOF Performance (calculated on combined OOF predictions):")
oof_metrics = {}
for model_name in model_names_with_preds:
     if model_name in oof_df.columns:
        rmse = np.sqrt(mean_squared_error(oof_df['True'], oof_df[model_name]))
        r2 = r2_score(oof_df['True'], oof_df[model_name])
        oof_metrics[model_name] = {'OOF_RMSE': rmse, 'OOF_R2': r2}
        print(f"  {model_name}: OOF RMSE={rmse:.4f}, OOF R2={r2:.4f}")


## 9. Ensemble Methods & Final Model Selection

In [ ]:
print("--- 9. Ensemble Methods (using OOF predictions) ---")

# Use the OOF predictions (oof_df) as input features for a meta-learner (Stacking)
# or simply average/weight the best models (Blending/Voting).

# --- 9a. Blending/Voting (Simpler) ---
# Select top N models based on OOF performance or CV results
top_models = nested_cv_results_df.head(3).index.tolist()
print(f"Selected top models for blending: {top_models}")

if len(top_models) > 1:
    # Simple Averaging Blend
    oof_df['Blend_Avg'] = oof_df[top_models].mean(axis=1)
    blend_rmse = np.sqrt(mean_squared_error(oof_df['True'], oof_df['Blend_Avg']))
    blend_r2 = r2_score(oof_df['True'], oof_df['Blend_Avg'])
    print(f"  Simple Average Blend: OOF RMSE={blend_rmse:.4f}, OOF R2={blend_r2:.4f}")

    # Weighted Average Blend (e.g., based on inverse RMSE)
    weights = 1 / nested_cv_results_df.loc[top_models, 'RMSE_mean']
    weights = weights / weights.sum() # Normalize
    oof_df['Blend_Weighted'] = np.average(oof_df[top_models], axis=1, weights=weights)
    blend_w_rmse = np.sqrt(mean_squared_error(oof_df['True'], oof_df['Blend_Weighted']))
    blend_w_r2 = r2_score(oof_df['True'], oof_df['Blend_Weighted'])
    print(f"  Weighted Average Blend: OOF RMSE={blend_w_rmse:.4f}, OOF R2={blend_w_r2:.4f}")

# --- 9b. Stacking ---
# Train a meta-model (e.g., Ridge) on OOF predictions
print("\nSetting up Stacking (conceptual - requires final model training):")
# 1. Select base models (can be different from top_models)
base_models_stacking = [(name, best_models_per_fold[f'Fold_{N_SPLITS_OUTER-1}'][name])
                        for name in top_models if name in best_models_per_fold[f'Fold_{N_SPLITS_OUTER-1}']]
# NOTE: This uses models trained on the LAST fold for illustration. A proper approach
#       would retrain final models on full dataset or use all fold models.

if len(base_models_stacking) > 1:
   # Define Meta Learner
   meta_learner = RidgeCV()

   # Stacking Regressor (conceptual setup)
   # stack = StackingRegressor(estimators=base_models_stacking, final_estimator=meta_learner, cv='prefit') # Needs careful handling of prefit models
   # A common approach is to train the meta-learner directly on the oof_df predictions
   X_meta_train = oof_df[top_models]
   y_meta_train = oof_df['True']

   meta_learner.fit(X_meta_train, y_meta_train)
   y_pred_stack_oof = meta_learner.predict(X_meta_train)
   stack_rmse_oof = np.sqrt(mean_squared_error(y_meta_train, y_pred_stack_oof))
   stack_r2_oof = r2_score(y_meta_train, y_pred_stack_oof)
   print(f"  Stacking (Meta-Learner trained on OOF): OOF RMSE={stack_rmse_oof:.4f}, OOF R2={stack_r2_oof:.4f}")
   # To get final predictions on new data, base models predict first, then meta-learner predicts on those outputs.
else:
    print(" Not enough base models available for stacking example.")

# --- Final Model Selection ---
# Choose the best single model or ensemble based on OOF performance and stability (CV std dev).
print("\nFinal Model: Select based on overall OOF performance (e.g., lowest RMSE from individual models or ensembles).")

## 10. Model Interpretation & Explainability

In [ ]:
print("--- 10. Model Interpretation (Example using SHAP) ---")

# Select the best performing model (or a tree-based model for easier SHAP interpretation)
# Retrain the final chosen model on the full training dataset (or use one from the last fold for illustration)
final_model_name = nested_cv_results_df.index[0] # Example: Best model from CV
print(f"Interpreting model: {final_model_name}")

# Need data in the format the model was trained on (after FE, FS, Scaling)
# Retrain final pipeline steps on full X, y (or use last fold's components for illustration)
# Placeholder: Assume we have a final fitted model 'final_model' and corresponding scaled full training data 'X_scaled_full'
# Example: Using the model from the last fold and the corresponding training data
try:
    last_fold_idx = N_SPLITS_OUTER - 1
    last_fold_key = f'Fold_{last_fold_idx+1}'
    final_model = best_models_per_fold[last_fold_key][final_model_name]

    # Re-run preprocessing steps on the *training data* of the last fold to get the input for SHAP
    train_outer_idx_last, _ = list(outer_cv.split(X, y, groups=cv_groups))[last_fold_idx]
    X_train_outer_last = X.iloc[train_outer_idx_last]
    y_train_outer_last = y[train_outer_idx_last]
    # Apply FE/FS/Scaling exactly as done in that fold's training
    # Placeholder for simplicity - using the scaled data from that fold
    # Need to regenerate X_train_outer_scaled from the loop above for the last fold
    # This part requires careful management of the processing pipeline state from the CV loop
    print("SHAP requires correctly processed data matching the model's training input.")
    print("Conceptual SHAP application:")

    # Choose appropriate explainer based on model type
    if isinstance(final_model, (RandomForestRegressor, GradientBoostingRegressor, xgb.XGBRegressor, lgb.LGBMRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor)):
        explainer = shap.TreeExplainer(final_model)
        # Need the corresponding X_train_outer_scaled for this fold
        # shap_values = explainer.shap_values(X_train_outer_scaled) # Placeholder data
    elif isinstance(final_model, (RidgeCV, LassoCV, SVR, KNeighborsRegressor)):
        # Use KernelExplainer (slower) - requires a background dataset
        # background_data = shap.sample(X_train_outer_scaled, 100) # Sample background data
        # explainer = shap.KernelExplainer(final_model.predict, background_data)
        # shap_values = explainer.shap_values(X_test_outer_scaled_from_last_fold) # Example on test data
        print(f"SHAP for {final_model_name} might require KernelExplainer (slower).")
    else:
        print(f"SHAP explainer not straightforwardly defined for {type(final_model)}.")

    # If SHAP values were calculated:
    # shap.summary_plot(shap_values, X_train_outer_scaled, feature_names=X_train_outer_sel.columns) # Use correct data/features
    # shap.summary_plot(shap_values, X_train_outer_scaled, plot_type='bar', feature_names=X_train_outer_sel.columns)
    # Can also plot dependence plots: shap.dependence_plot('feature_name', shap_values, X_train_outer_scaled)
except KeyError as e:
     print(f"Could not retrieve model/data for SHAP for {final_model_name} from last fold: {e}")
except Exception as e:
     print(f"An error occurred during SHAP setup: {e}")

## 11. Reporting, Discussion & Future Work

In [ ]:
print("--- 11. Final Report Summary ---")

display(Markdown("### Final Results Summary"))
print("Nested CV Aggregated Performance:")
display(nested_cv_results_df.round(4))
print("\nOverall OOF Performance (using combined predictions):")
display(pd.DataFrame(oof_metrics).T.round(4))
# Add Ensemble OOF results if calculated

display(Markdown("### Discussion"))
display(Markdown(f"""
*   **Best Performing Approach:** Based on Nested CV (RMSE mean: {nested_cv_results_df.iloc[0]['RMSE_mean']:.4f} +/- {nested_cv_results_df.iloc[0]['RMSE_std']:.4f}) and OOF validation (RMSE: {oof_metrics.get(nested_cv_results_df.index[0], {'OOF_RMSE': np.nan})['OOF_RMSE']:.4f}), the best approach identified in this analysis was **{nested_cv_results_df.index[0]}**.
*   **Impact of Feature Engineering/Selection:** Discuss which FE/FS choices were tested and their apparent impact (e.g., 'Using Lasso selection resulted in slightly better performance for tree models compared to no selection...').
*   **Model Comparison:** Compare the performance of different model families (linear, trees, SVM, DL). Were tree ensembles consistently better? How did LSTM fare?
*   **Ensemble Performance:** Did blending or stacking provide a significant improvement over the best single model?
*   **Interpretability:** Summarize findings from SHAP analysis (if performed). Which features consistently drove predictions? Do they align with clinical knowledge or findings from the literature review?
"""))

display(Markdown("### Limitations"))
display(Markdown("""
*   **Pre-extracted Features:** The primary limitation is the lack of raw audio data. Signal processing features (FFT, Wavelets) were applied to the *feature time series*, not the original voice signal, limiting their physical interpretability and potential effectiveness compared to analysis of raw audio.
*   **Dataset Size/Diversity:** The dataset size and number of subjects might limit the generalizability of findings.
*   **Feature Engineering Scope:** The tested FE techniques represent a subset of possibilities.
*   **Hyperparameter Search Space:** The Bayesian optimization explored a defined space; wider or different spaces might yield better results.
*   **Computational Resources:** Exhaustive search across all combinations of FE, FS, HPO is computationally prohibitive.
*   **Validation Strategy:** While Nested CV with GroupKFold is robust, it assumes subject labels are correct and fully capture dependencies.
"""))

display(Markdown("### Future Work"))
display(Markdown("""
*   **Acquire Raw Audio:** If possible, repeating the analysis starting from raw audio signals would be the most impactful next step, allowing for true signal processing FE.
*   **Explore More Features:** Investigate other feature types (e.g., fractal dimensions, entropy measures, phoneme-specific features if transcription possible).
*   **Advanced Deep Learning:** Experiment with more complex DL architectures (CNN-LSTM, Transformers, Attention mechanisms) if computationally feasible.
*   **Refined Subject Handling:** If more detailed subject metadata were available (e.g., demographics, medication status), incorporate it into the analysis or stratified validation.
*   **External Validation:** Test the final selected model on an independent Parkinson's voice dataset if available.
*   **Causality Analysis:** Explore causal inference techniques to understand feature relationships beyond correlation (requires strong assumptions).
*   **Longitudinal Analysis:** If data represented multiple recordings over time *for the same individuals*, model the progression trajectory rather than just a single score prediction.
"""))